In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
df = pd.read_csv('hospital_outpatient_DataSet.csv')

print("--- TASK 1: DATA AUDIT ---")
print(f"Dataset Shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())
print("\nData Info:")
df.info()

print("\nMissing Values Before Cleaning:")
print(df.isna().sum().sort_values(ascending=False))

print(f"\nDuplicate Count: {df.duplicated().sum()}")

--- TASK 1: DATA AUDIT ---
Dataset Shape: (300, 13)

First 5 rows:
  Visit_ID        Date   Department Doctor_Name  Patient_Age Gender Symptoms  \
0    V1170  01-01-2024   Cardiology   Dr. Ahmed         61.0      F  Allergy   
1    V1221  01-01-2024   Cardiology   Dr. Ahmed         67.0      M  Fatigue   
2    V1016  01-01-2024   Cardiology   Dr. Meena         48.0      F  Allergy   
3    V1205  03-01-2024  Orthopedics   Dr. Meena         34.0      F      NaN   
4    V1212  03-01-2024  Orthopedics   Dr. Meena         29.0      F    Fever   

      Diagnosis  Medicine_Cost  Consultation_Fee Followup_Required  \
0  Hypertension         1674.0             263.0                No   
1      Migraine          764.0             359.0               Yes   
2      Fracture         1085.0             727.0               Yes   
3  Hypertension         1893.0             323.0                No   
4     Infection         1056.0             357.0               Yes   

   Visit_Duration  Total_Bill  

In [5]:
# ==========================================
# 2) Cleaning (6 Actions)
# ==========================================
# Action 1: Fix datatype - Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')

# Action 2: Handle missing numeric values using Median
df['Patient_Age'] = df['Patient_Age'].fillna(df['Patient_Age'].median())
df['Medicine_Cost'] = df['Medicine_Cost'].fillna(df['Medicine_Cost'].median())
df['Consultation_Fee'] = df['Consultation_Fee'].fillna(df['Consultation_Fee'].median())

# Action 3: Handle missing categorical values using 'Unknown' or Mode
df['Symptoms'] = df['Symptoms'].fillna('Unknown')
df['Diagnosis'] = df['Diagnosis'].fillna('Unknown')
df['Followup_Required'] = df['Followup_Required'].fillna(df['Followup_Required'].mode()[0])
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])

# Action 4: Text cleaning - Strip whitespace and ensure consistent casing
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()

# Action 5: Handle duplicates
df = df.drop_duplicates()

# Action 6 (Extra): Ensure Department names are consistently Title Cased
df['Department'] = df['Department'].str.title()

print("\nMissing Values After Cleaning:", df.isna().sum().sum())


Missing Values After Cleaning: 0


In [6]:
# ==========================================
# 3) Create 3 New Columns (Feature Engineering)
# ==========================================
# Column 1: Conditional column (High Bill Flag)
df['High_Bill_Flag'] = np.where(df['Total_Bill'] > 2000, 'High', 'Normal')

# Column 2: Derived numeric column (Sum of costs)
df['Bill_Check'] = df['Medicine_Cost'] + df['Consultation_Fee']

# Column 3: Bucket column (Age Group)
df['Age_Group'] = pd.cut(df['Patient_Age'], 
                        bins=[0, 18, 40, 60, 100], 
                        labels=['Minor', 'Young Adult', 'Middle Aged', 'Senior'])

In [7]:
# ==========================================
# 4) Insights
# ==========================================
print("\n--- TASK 4: INSIGHTS ---")
# Insight 1: Average bill by Department
insight1 = df.groupby('Department')['Total_Bill'].agg(['mean', 'count']).sort_values(by='mean', ascending=False)
print("1. Avg Bill per Dept:\n", insight1)
print("Conclusion: The General department has the highest average patient bill.")

# Insight 2: Top Doctors by Visit Count
insight2 = df['Doctor_Name'].value_counts().head(3)
print("\n2. Top 3 Doctors:\n", insight2)
print("Conclusion: Dr. Ahmed and Dr. Meena are the busiest doctors in the facility.")

# Insight 3: Follow-up Requirement Distribution
insight3 = df['Followup_Required'].value_counts(normalize=True)
print("\n3. Follow-up Rate:\n", insight3)
print("Conclusion: Approximately 44% of all outpatient visits require a follow-up appointment.")


--- TASK 4: INSIGHTS ---
1. Avg Bill per Dept:
                     mean  count
Department                     
General      1634.482143     56
Orthopedics  1627.641509     53
Cardiology   1611.461538     65
Ent          1606.116883     77
Dermatology  1535.448980     49
Conclusion: The General department has the highest average patient bill.

2. Top 3 Doctors:
 Doctor_Name
Dr. Ahmed    67
Dr. Meena    65
Dr. Rao      61
Name: count, dtype: int64
Conclusion: Dr. Ahmed and Dr. Meena are the busiest doctors in the facility.

3. Follow-up Rate:
 Followup_Required
No     0.556667
Yes    0.443333
Name: proportion, dtype: float64
Conclusion: Approximately 44% of all outpatient visits require a follow-up appointment.


In [8]:
# ==========================================
# 5) Regression: Predict Total_Bill
# ==========================================
print("\n--- TASK 5: REGRESSION ---")
X_reg = df[['Department', 'Patient_Age', 'Medicine_Cost', 'Consultation_Fee', 'Visit_Duration']]
y_reg = df['Total_Bill']

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Pipeline for Regression
reg_preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['Patient_Age', 'Medicine_Cost', 'Consultation_Fee', 'Visit_Duration']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Department'])
])

reg_pipeline = Pipeline(steps=[
    ('preprocessor', reg_preprocessor),
    ('regressor', LinearRegression())
])

reg_pipeline.fit(X_train_r, y_train_r)
mae = mean_absolute_error(y_test_r, reg_pipeline.predict(X_test_r))
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print("MAE Explanation: This represents the average dollar difference between our predicted bills and actual costs.")


--- TASK 5: REGRESSION ---
Mean Absolute Error (MAE): 36.12
MAE Explanation: This represents the average dollar difference between our predicted bills and actual costs.


In [9]:
# ==========================================
# 6) Classification: Predict Followup_Required
# ==========================================
print("\n--- TASK 6: CLASSIFICATION ---")
X_clf = df[['Department', 'Gender', 'Symptoms', 'Diagnosis', 'Total_Bill', 'Visit_Duration']]
y_clf = df['Followup_Required'].map({'Yes': 1, 'No': 0})

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# Pipeline for Classification
clf_preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['Total_Bill', 'Visit_Duration']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Department', 'Gender', 'Symptoms', 'Diagnosis'])
])

clf_pipeline = Pipeline(steps=[
    ('preprocessor', clf_preprocessor),
    ('classifier', LogisticRegression())
])

clf_pipeline.fit(X_train_c, y_train_c)
y_pred_c = clf_pipeline.predict(X_test_c)

print("Confusion Matrix:\n", confusion_matrix(y_test_c, y_pred_c))
print(f"Precision: {precision_score(y_test_c, y_pred_c):.2f}")
print(f"Recall: {recall_score(y_test_c, y_pred_c):.2f}")
print("Recall Importance: High recall ensures we don't miss patients who truly need follow-up care.")


--- TASK 6: CLASSIFICATION ---
Confusion Matrix:
 [[22 14]
 [16  8]]
Precision: 0.36
Recall: 0.33
Recall Importance: High recall ensures we don't miss patients who truly need follow-up care.


In [10]:
# ==========================================
# 7) Final Conclusion
# ==========================================
print("\n--- TASK 7: FINAL CONCLUSION ---")
"""
The data analysis reveals that the General department drives the highest revenue per visit, 
while Dr. Ahmed manages the highest patient volume. Our Linear Regression model achieved 
an MAE of approximately 36, indicating highly accurate billing predictions based on costs. 
The Classification model showed a recall of 0.33, suggesting room for improvement in 
identifying all follow-up candidates. A key limitation is the small dataset size (300 records), 
which may limit the generalizability of the Logistic Regression model.
"""


--- TASK 7: FINAL CONCLUSION ---


'\nThe data analysis reveals that the General department drives the highest revenue per visit, \nwhile Dr. Ahmed manages the highest patient volume. Our Linear Regression model achieved \nan MAE of approximately 36, indicating highly accurate billing predictions based on costs. \nThe Classification model showed a recall of 0.33, suggesting room for improvement in \nidentifying all follow-up candidates. A key limitation is the small dataset size (300 records), \nwhich may limit the generalizability of the Logistic Regression model.\n'

In [11]:
# Export the cleaned dataframe to a new CSV file
topath=r"C:\Users\vaish\Downloads\cleaned_hospital_outpatient_data.xlsx"
df.to_csv(topath,index=False)